## Árbol de decisión

Este modelo busca predecir el cargo que tiene un servicio depéndiendo del trafico
* congestion_surcharge = 0 ---> sin trafico
* congestion_surcharge = 2 ---> con trafico

Se importan las librerias requeridas

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from joblib import dump

Se hace el entrenamiento con 3 datasets de recorridos de taxis ecologicos para tres meses consecutivos del año 2023, se concatenan y se usan para el entrenamiento

In [3]:
d0 = pd.read_parquet("C:/Users/Acer/Downloads/green_tripdata_2023-12_limpio.parquet")
df1 = pd.read_parquet("C:/Users/Acer/Downloads/green_tripdata_2023-11_limpio.parquet")
df2 = pd.read_parquet("C:/Users/Acer/Downloads/green_tripdata_2023-10_limpio.parquet")
df = pd.concat([d0, df1, df2])

In [4]:
df.shape

(183020, 12)

Se filtra el data set con los valores de la columna 'congestion_surcharge' mayor que -1 ya que -1 representa a los valores nulos

In [5]:
df = df[df['congestion_surcharge'] > -1]
df.shape

(169725, 12)

In [6]:
df['congestion_surcharge'].value_counts()

congestion_surcharge
0    120670
2     49055
Name: count, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169725 entries, 0 to 61499
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   start_trip            169725 non-null  datetime64[us]
 1   end_trip              169725 non-null  datetime64[us]
 2   pu_location_id        169725 non-null  int64         
 3   do_location_id        169725 non-null  int64         
 4   passenger_count       169725 non-null  int64         
 5   trip_distance         169725 non-null  float64       
 6   fare_amount           169725 non-null  float64       
 7   tip_amount            169725 non-null  float64       
 8   total_amount          169725 non-null  float64       
 9   payment_type          169725 non-null  int64         
 10  congestion_surcharge  169725 non-null  int64         
 11  type_of_taxi          169725 non-null  int64         
dtypes: datetime64[us](2), float64(4), int64(6)
memory usage: 16.8 MB

In [8]:
df.head(1)

,start_trip,end_trip,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,congestion_surcharge,type_of_taxi
0,2023-12-01 00:27:37,2023-12-01 00:42:48,74,243,1,4.8,22.6,5.02,30.12,1,0,1


Se extrate informacion de las fechas de inicio y fin de viaje para generar columnas con los siguientes valores: año, mes, dia y minutos de viaje

In [9]:
df['year'] = df['start_trip'].dt.year
df['month'] = df['start_trip'].dt.month
df['day'] = df['start_trip'].dt.dayofweek
df['min_trip'] = df['min_trip'] = (df['end_trip'] - df['start_trip']).dt.total_seconds() / 60
df.sample(5)

,start_trip,end_trip,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,congestion_surcharge,type_of_taxi,year,month,day,min_trip
56394,2023-10-29 12:29:18,2023-10-29 12:36:06,75,41,2,1.40,9.3,2.16,12.96,1,0,1,2023,10,6,6.800000
58988,2023-12-31 19:20:24,2023-12-31 19:33:14,66,33,1,1.41,12.8,2.86,17.16,1,0,1,2023,12,6,12.833333
10811,2023-12-06 13:32:21,2023-12-06 13:41:59,74,236,1,1.98,11.4,3.13,18.78,1,2,1,2023,12,2,9.633333
57668,2023-10-30 08:27:57,2023-10-30 08:42:51,95,121,1,2.40,16.3,0.00,17.80,-1,0,1,2023,10,0,14.900000
5099,2023-11-03 09:21:05,2023-11-03 09:32:16,95,95,1,1.54,12.1,0.00,13.60,-1,0,1,2023,11,4,11.183333


In [10]:
df.columns

Index(['start_trip', 'end_trip', 'pu_location_id', 'do_location_id',
       'passenger_count', 'trip_distance', 'fare_amount', 'tip_amount',
       'total_amount', 'payment_type', 'congestion_surcharge', 'type_of_taxi',
       'year', 'month', 'day', 'min_trip'],
      dtype='object')

In [11]:
df.shape

(169725, 16)

In [12]:
# Arbol de decision

In [13]:
from sklearn import tree
# Selección de las variables independientes
X = df[['min_trip', 'trip_distance', 'pu_location_id', 'do_location_id']]

# Conversión de las variables categóricas a dummies
X = pd.get_dummies(X)

# Selección de la variable dependiente
y = df['congestion_surcharge']

# Separación del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Creación del clasificador de árbol de decisión
arbol = tree.DecisionTreeClassifier(max_depth = 7, random_state=42)

# Entrenamiento del modelo
arbol.fit(X_train, y_train)

# Evaluación del modelo
score = arbol.score(X_test, y_test)

# Predicción del recargo por congestión
predicciones = arbol.predict(X_test)

# Impresión del score del modelo
print(f"Score del modelo: {score}")

# Impresión de las primeras 10 predicciones
#print(f"Primeras 10 predicciones: {predicciones[:10]}")

# Visualización del árbol de decisión
#tree.plot_tree(arbol)

Score del modelo: 0.9317967571644042


In [15]:
resultados = pd.DataFrame({'Valor real': y_test, 'Valor predicho': predicciones}).head(5)

# Impresión del dataframe
print(resultados.to_string())

       Valor real  Valor predicho
26660           0               0
41798           0               0
3571            2               2
38150           2               2
38007           0               0


Se observa un Score de 0.93, y se visualiza una columna con valores reales y valores predichos

Se guarda el modelo en formato joblib, para ser posteriormente consumido por Streamlit

In [ ]:
dump(arbol, 'E:/proyecto_final/Modelo/prediccion_congestion.joblib')

['E:/proyecto_final/Modelo/prediccion_congestion.joblib']